In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import socket

import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *#avg, count, expr
from pyspark.sql.types import *

In [26]:
# initialize
sc = pyspark.SparkContext()
spark = SparkSession(sc)
spark.sparkContext.appName = 'streaming'
# show the number of cores
print('%d cores'%spark._jsc.sc().getExecutorMemoryStatus().keySet().size())
spark

1 cores


In [27]:
# setup to read the tweets from the socket
listenHost = '127.0.0.1' 
listenPort = 4205
tweets = spark.readStream.format('socket').options(host=listenHost, port=listenPort).load()

In [28]:
# start the streaming
writeTweet = tweets.writeStream.outputMode('append').format('memory').queryName('tweetquery').trigger(processingTime='2 seconds').start()
print('Tweets streaming')

Tweets streaming


In [30]:
spark.sql('select * from tweetquery').show()

+-----+
|value|
+-----+
+-----+



In [32]:
writeTweet.stop()

Py4JJavaError: An error occurred while calling o244.stop.
: java.lang.IllegalStateException: Cannot call methods on a stopped SparkContext.
This stopped SparkContext was created at:

org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
py4j.GatewayConnection.run(Gate

In [31]:
sc.stop()